# Named Entity Recognition (NER)

Dis notebook na from [AI for Beginners Curriculum](http://aka.ms/ai-beginners).

For dis example, we go learn how to train NER model on [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) Dataset wey dey Kaggle. Before you continue, abeg download [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) file put for di current directory.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## How to Prepare di Dataset

We go first read di dataset put for inside one dataframe. If you wan sabi more about how to use Pandas, check dis [lesson on data processing](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) for our [Data Science for Beginners](http://aka.ms/datascience-beginners).


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Make we get di unique tags and create lookup dictionaries wey we fit use change tags to class numbers:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Now we need to do di same wit vocabulary. To make am simple, we go create vocabulary witout dey check word frequency; for real life, you fit wan use Keras vectorizer, and limit di number of words.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

We need to create dataset of sentences for training. Make we loop through di original dataset and separate all di individual sentences into `X` (lists of words) and `Y` (list of tokens):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


We go now change all words and tokens to vector:


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

To make am easy, we go add 0 tokens for all sentences reach di maximum length. For real life, we fit wan use beta strategy, and add padding only for one minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Define Token Classification Network

We go use two-layer bidirectional LSTM network for token classification. To fit dense classifier for each output wey dey last LSTM layer, we go use `TimeDistributed` construction, wey dey replicate di same dense layer for each output of LSTM for each step:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Make sure say we dey talk say `maxlen` for our dataset - if we wan make di network fit handle sequence wey get different length, we go need use brain well well when we dey define di network.

Make we train di model now. For quick quick, we go only train am for one epoch, but you fit try train am for longer time. Plus, e go make sense if you fit divide some part of di dataset as training dataset, so you fit check validation accuracy.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Testing di Result

Make we see how our entity recognition model go work for one sample sentence:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Takeaway

Even simple LSTM model dey show beta result for NER. But, if you wan get better result, you fit use big pre-trained language models like BERT. How to train BERT for NER wit Huggingface Transformers library dey described [here](https://huggingface.co/course/chapter7/2?fw=pt).


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don translate wit AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). Even as we dey try make am accurate, abeg sabi say machine translation fit get mistake or no dey correct well. Di original dokyument for im native language na di main source wey you go fit trust. For important mata, e better make professional human translation dey use. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen from di use of dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
